In [15]:
from ultralytics import YOLO
import os
import random
import cv2
import numpy as np
import shutil


In [16]:
# Last inn en forhåndstrent YOLO-modell (anbefales for trening)
model = YOLO('yolov8n.pt')


In [17]:
# Tren modellen ved å bruke 'data.yaml' datasettet i 3 epoker
results = model.train(data='data.yaml', epochs=3, batch=8, imgsz=(2560, 1440), project='/work/mbergst/project/ultralytics/runs/detect')



New https://pypi.org/project/ultralytics/8.1.45 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=3, time=None, patience=100, batch=15, imgsz=(1600, 900), save=True, save_period=-1, cache=False, device=None, workers=8, project=/work/mbergst/project/ultralytics/runs/detect, name=train24, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, ret

train: Scanning /work/mbergst/project/data_yolov8/1_train-val_1min_aalesund_from_start/labels/train.cache... 1441 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1441/1441 [00:00<?, ?it/s]
val: Scanning /work/mbergst/project/data_yolov8/1_train-val_1min_aalesund_from_start/labels/val.cache... 361 images, 0 backgrounds, 0 corrupt: 100%|██████████| 361/361 [00:00<?, ?it/s]


Plotting labels to /work/mbergst/project/ultralytics/runs/detect/train24/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00046875), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1600 train, 1600 val
Using 8 dataloader workers
Logging results to /work/mbergst/project/ultralytics/runs/detect/train24
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/3      19.1G     0.9067      1.227     0.8524         22       1600: 100%|██████████| 97/97 [00:19<00:00,  4.90it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:02<00:00,  4.73it/s]

                   all        361       8361      0.995      0.487      0.522      0.426


...

        3/3      17.9G     0.7061     0.5654     0.8193         22       1600: 100%|██████████| 97/97 [00:19<00:00,  5.06it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:01<00:00,  7.59it/s]

                   all        361       8361       0.91      0.745      0.793      0.587



3 epochs completed in 0.019 hours.
Optimizer stripped from /work/mbergst/project/ultralytics/runs/detect/train24/weights/last.pt, 6.4MB
Optimizer stripped from /work/mbergst/project/ultralytics/runs/detect/train24/weights/best.pt, 6.4MB

Validating /work/mbergst/project/ultralytics/runs/detect/train24/weights/best.pt...
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:02<00:00,  4.64it/s]


                   all        361       8361      0.908      0.748      0.793      0.587
                  ball        361        241      0.851      0.499       0.59      0.329
                player        361       8120      0.964      0.997      0.995      0.845
Speed: 0.2ms preprocess, 1.2ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to /work/mbergst/project/ultralytics/runs/detect/train24


In [18]:
# Evaluér modellens prestasjon på valideringssettet
val_results = model.val()


Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /work/mbergst/project/data_yolov8/1_train-val_1min_aalesund_from_start/labels/val.cache... 361 images, 0 backgrounds, 0 corrupt: 100%|██████████| 361/361 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:03<00:00,  6.48it/s]


                   all        361       8361      0.914      0.749      0.795      0.607
                  ball        361        241      0.864      0.501      0.596      0.359
                player        361       8120      0.964      0.997      0.995      0.856
Speed: 0.3ms preprocess, 2.7ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to /work/mbergst/project/ultralytics/runs/detect/train242


In [19]:
# Test modellen på testsettet
test_results = model.val(data='test.yaml')

Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)


val: Scanning /work/mbergst/project/data_yolov8/3_test_1min_hamkam_from_start/labels/test.cache... 1802 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1802/1802 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 121/121 [00:13<00:00,  8.71it/s]


                   all       1802      42192      0.658      0.638      0.613      0.465
                  ball       1802       1412      0.372      0.317      0.254      0.157
                player       1802      40780      0.944      0.959      0.972      0.773
Speed: 0.3ms preprocess, 2.2ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to /work/mbergst/project/ultralytics/runs/detect/train243
